In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)

In [ ]:
#
# This cell is to be put in python/flashmatch/visualization/view_api.py
# Note import for visualization is different, though, for vis_api.py 
#
import numpy as np
import plotly.graph_objs as go
from flashmatch import flashmatch, AnalysisManager
from flashmatch.visualization import vis_icarus, icarus_layout3d

class DataManager:
    """
    DataManager class "holds" data retrieved using AnalysisManager. In particular, it holds:
      - "current" entry and event id
      - "cpp" QCluster,Flash_t, etc. in flashmatch.FlashMatchInput format (this is self.cpp attribute)
      - numpy array representation of what's stored in "cpp"
      - Read & update local data attributes is done via self.update function
    TODO: useful properties of DataManager is incremental to AnalysisManager. One can
          either merge two classes or make DataManager inherit from AnalysisManager, if wished.
    """
    def __init__(self):
        self.entry = -1
        self.event = -1
        self.hypothesis_made = False
        
    def update(self,manager,entry,is_entry=True,make_hypothesis=False):
        """
        Given flashmatch.AnalysisManager, which can interface input data stream and OpT0Finder tools,
        this function "read and update" local data attributes if needed.
        """
        if not is_entry:
            entry = manager.entry_id(entry)
        if entry < 0 or entry >= len(manager.entries()): return
        if not self.entry == entry:
            self.cpp = manager.make_flashmatch_input(entry)
            self.np_qcluster_v = [flashmatch.as_ndarray(qcluster) for qcluster in self.cpp.qcluster_v]
            self.np_flash_v    = [flashmatch.as_ndarray(flash)    for flash    in self.cpp.flash_v   ]
            self.entry = entry
            self.event = manager.event_id(entry)
            self.hypothesis_made = False
        if make_hypothesis and not self.hypothesis_made:
            self.cpp.hypo_v = [manager.flash_hypothesis(qcluster) for qcluster in self.cpp.qcluster_v]
            self.np_hypo_v  = [flashmatch.as_ndarray(flash)       for flash    in self.cpp.hypo_v    ]
            for idx in range(len(self.np_hypo_v)):
                self.cpp.hypo_v[idx].idx = self.cpp.qcluster_v[idx].idx

class AppManager:
    """
    AppManager serves 2 purposes: 0) it holds data objects which state must be tracked
    through the lifetime of an (stateless) dash html app, and 1) it implements useful
    data parsing methods (e.g. creating a figure, update dropdown options, etc.).
    """
    def __init__(self,cfg,geo,data_particle,data_opflash):
        """
        INPUT:
          - cfg ... OpT0Finder configuration file, passed onto AnalysisManager
          - geo ... vis_icarus (geometry) data file in either PSet or yaml format
          - data_particle ... particle data file stored by ICARUSParticleAna_module
          - data_opflash ... OpFlash data file stored by ICARUSOpFlashAna_module
        """
        self.ana_manager = AnalysisManager(cfg=cfg, particleana=data_particle, opflashana=data_opflash)
        assert(len(self.ana_manager.entries()))
        self.dat_manager = DataManager()
        self.vis_icarus = vis_icarus(geo)
        self.detector_trace = self.vis_icarus.get_trace_detector()
        self.detector_trace_no_pmt = self.vis_icarus.get_trace_detector(draw_pmts=False)
        self.layout = icarus_layout3d(self.vis_icarus.data(),set_camera=False,dark=True)
        self.empty_view = go.Figure(self.detector_trace,layout=self.layout)
        
    def current_data_index(self,is_entry):
        """
        Returns current "data index"
        INPUTS:
          - is_entry ... a boolean True=return entry, False=return event id
        OUTPUT:
          - data index integer, either entry or event id
        """
        return self.dat_manager.entry if is_entry else self.dat_manager.event
    
    def dropdown_qcluster(self,data_index,is_entry):
        """
        Generates a dropdown menu for a dash app to select a QCluster
        INPUTS:
          - data_index ... an integer to specify which entry/event to read
          - is_entry ... True=entry, False=event id
        OUTPUT:
          A list of dicts to be consumed by dash app dropdown options
        """
        self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry)
        idx_v = [qcluster.idx for qcluster in self.dat_manager.cpp.qcluster_v]
        dropdown_qcluster = [dict(label='Track %02d (%d pts)' % (idx_v[idx],len(qcluster)), 
                                  value=idx) 
                             for idx,qcluster in enumerate(self.dat_manager.np_qcluster_v)]
        dropdown_qcluster += [dict(label='All tracks',value=len(self.dat_manager.np_qcluster_v))]
        return dropdown_qcluster
    
    def dropdown_flash(self,data_index,is_entry,is_hypothesis):
        """
        Generates a dropdown menu for a dash app to select a Flash
        INPUTS:
          - data_index ... an integer to specify which entry/event to read
          - is_entry ... True=entry, False=event id
        OUTPUT:
          A list of dicts to be consumed by dash app dropdown options
        """
        self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry, make_hypothesis=is_hypothesis)
        target_v = self.dat_manager.np_flash_v if not is_hypothesis else self.dat_manager.np_hypo_v
        idx_v  = [flash.idx for flash in self.dat_manager.cpp.flash_v]
        time_v = np.zeros(shape=[len(target_v)])
        if not is_hypothesis: time_v = [flash.time for flash in self.dat_manager.cpp.flash_v]
        dropdown_flash  = [dict(label='Flash %02d (t=%f us  PE=%d)' % (idx_v[idx],
                                                                       time_v[idx],
                                                                       int(flash.sum()/100.)
                                                                      ),
                                value=idx)
                            for idx,flash in enumerate(target_v)]
        dropdown_flash += [dict(label='All flashes',value=len(target_v))]
        return dropdown_flash
    
    def event_display(self, data_index, qcluster_idx_v, flash_idx_v, is_entry, is_hypothesis):
        """
        Generate 3D display for change in event/entry and/or selection of flash/qcluster
        """
        self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry, make_hypothesis=is_hypothesis)
        data = []
        # QCluster
        if qcluster_idx_v is not None and len(qcluster_idx_v):
            if len(self.dat_manager.np_qcluster_v) in qcluster_idx_v:
                qcluster_idx_v = range(len(_blob.np_qcluster_v))
            idx_v = [self.dat_manager.cpp.qcluster_v[idx].idx for idx in qcluster_idx_v]
            for idx in qcluster_idx_v:
                xyz = self.dat_manager.np_qcluster_v[idx]
                name = 'Track %02d (%d pts)' % (self.dat_manager.cpp.qcluster_v[idx].idx,len(xyz))
                trace = go.Scatter3d(x=xyz[:,0],y=xyz[:,1],z=xyz[:,2],mode='markers',
                                     name=name,
                                     marker = dict(size=2, opacity=0.5)
                                    )
                data.append(trace)
        target_v = self.dat_manager.np_flash_v if not is_hypothesis else self.dat_manager.np_hypo_v
        if flash_idx_v and len(flash_idx_v):
            pmt_pos = self.vis_icarus.data()['pmts']
            pmt_val = None
            if len(target_v) in flash_idx_v: pmt_val = np.sum(target_v,axis=0)
            else: pmt_val = np.sum(np.column_stack([target_v[idx] for idx in flash_idx_v]),axis=1)
            trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                 name='Flash (%f PEs)' % np.sum(pmt_val),
                                 marker = dict(size=6, color=pmt_val, opacity=0.5)
                                )
            data.append(trace)
        elif len(data):
            # no pmt data => show default pmt view
            return go.Figure(self.detector_trace + data, layout=self.layout)

        if len(data)<1:
            return self.empty_view
        else:
            return go.Figure(self.detector_trace_no_pmt + data, layout=self.layout)

    def hypothesis_display(self, data_index, qcluster_idx_v, is_entry, xoffset):
        """
        Generate 3D display of a flash hypothesis for a given QCluster and x-offset
        """
        self.dat_manager.update(self.ana_manager, entry=data_index, is_entry=is_entry, make_hypothesis=False)
        data = []
        if qcluster_idx_v is None or len(qcluster_idx_v)<1: return self.empty_view
        if len(self.dat_manager.np_qcluster_v) in qcluster_idx_v:
            qcluster_idx_v = range(len(self.dat_manager.np_qcluster_v))
        qcluster = flashmatch.QCluster_t()
        for idx in qcluster_idx_v:
            qcluster += self.dat_manager.cpp.qcluster_v[idx]
        # find x span allowed
        xmin,xmax=qcluster.min_x(),qcluster.max_x()
        # if xoffset is larger than xmax-xmin, truncate
        xoffset = min(xoffset,xmax-xmin)
        # shift
        qcluster += (xoffset - xmin)
        # make hypothesis
        hypothesis = self.ana_manager.flash_hypothesis(qcluster)
        # make a trace for tpc
        xyzv = flashmatch.as_ndarray(qcluster)
        tpc_trace = go.Scatter3d(x=xyzv[:,0],y=xyzv[:,1],z=xyzv[:,2],mode='markers',
                                 name='Track (offset %f)' % xoffset,
                                 marker = dict(size=2,opacity=0.5)
                                )
        # make a trace for pmt
        pmt_val = flashmatch.as_ndarray(hypothesis)
        pmt_pos = self.vis_icarus.data()['pmts']
        pmt_trace = go.Scatter3d(x=pmt_pos[:,0],y=pmt_pos[:,1],z=pmt_pos[:,2],mode='markers',
                                 name='Hypothesis (%f PEs)' % np.sum(pmt_val),
                                 marker = dict(size=6,
                                               color=pmt_val,
                                               colorscale=None,
                                               opacity=0.5)
                                )
        return go.Figure(self.detector_trace_no_pmt + [pmt_trace] + [tpc_trace], layout=self.layout)
        

In [ ]:
#
# This cell is to be put in python/flashmatch/visualization/view_data.py
#
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

def view_data(cfg,geo,data_particle,data_opflash):
    """
    Visualized OpT0Finder input data as well as FlashHypothesis for varying x-position
    INPUT:
      - cfg ... OpT0Finder configuration file, passed onto AnalysisManager
      - geo ... vis_icarus (geometry) data file in either PSet or yaml format
      - data_particle ... particle data file stored by ICARUSParticleAna_module
      - data_opflash ... OpFlash data file stored by ICARUSOpFlashAna_module
    """
    _manager = AppManager(cfg=cfg, geo=geo, data_particle=data_particle, data_opflash=data_opflash)
    entry = 0
    xmin = _manager.vis_icarus.data()['tpc0'][0][0]
    xmax = _manager.vis_icarus.data()['tpc1'][1][0]

    #
    # Create app
    #
    #app = dash.Dash('Flash Match Event Viewer')#,external_stylesheets=[dbc.themes.DARKLY])
    app = dash.Dash('Flash Match Event Viewer',
                    external_stylesheets= ['https://cdnjs.cloudflare.com/ajax/libs/animate.css/3.5.2/animate.min.css)']
                   )
    header_text_style = dict(fontFamily='Georgia', fontWeight=300, fontSize=26)
    label_text_style = dict(fontFamily='Georgia', fontWeight=300, fontSize=20)

    app.layout = html.Div([
        html.H2('Flash Matching Data Viewer', style=header_text_style),
        
        html.Div([html.Label('View data entry: (0-%d)' % (len(_manager.ana_manager.entries())-1),
                             style=label_text_style)
                 ],style={'padding': '5px'}),
        
        html.Div([dcc.RadioItems(id='entry_or_event',
                                 options=[dict(label='Entry Number',value='entry'),
                                          dict(label='Event ID',value='event')],
                                 value='entry'),
                  dcc.Input(id='data_index', value=str(entry), type='int')],
                 style={'width': '100%', 'display': 'inline-block', 'padding': '5px'}),
        
        html.Div([html.Label('View PMTs with OpFlash or Hypothesis?',
                             style=label_text_style),
                  dcc.RadioItems(id='flash_or_hypo',
                                 options=[dict(label='Flash',value='flash'),
                                          dict(label='Hypothesis',value='hypothesis')],
                                 value='flash'),],
                 style={'width': '50%', 'display': 'inline-block', 'padding': '5px'}),
        dcc.Loading(id="loading", children=[html.Div(id="wait_out")], type="default"),
        html.Div([html.Label('Select QCluster_t to display', style=label_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='select_qcluster',
                               options=_manager.dropdown_qcluster(entry,is_entry=True),
                               multi=True),
                 ],style={'padding': '5px'}),
        
        html.Div([html.Label('Select Flash_t to display', style=label_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='select_flash',
                               options=_manager.dropdown_flash(entry,is_entry=True,is_hypothesis=False),
                               multi=True),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Graph(id='visdata',figure=_manager.empty_view)],style={'padding': '5px'}),
        # Hypothesis event display
        html.H2('Hypothesis playground: move QCluster along x!',style=header_text_style),
        
        html.Div([html.Label('X Offset', style=label_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Slider(id='xoffset',min=xmin,max=xmax,step=1.,value=xmin),
                 ],style={'padding': '5px'}),
        
        html.Div([html.Label('Select QCluster to display', style=label_text_style),
                 ],style={'padding': '5px'}),
        html.Div([dcc.Dropdown(id='target_qcluster',
                               options=_manager.dropdown_qcluster(entry,is_entry=True),
                               multi=True),
                 ],style={'padding': '5px'}),
        #dcc.Loading(id="loading-playdata",
        #            children=[html.Div([dcc.Graph(id='playdata',figure=_manager.empty_view)],
        #                               style={'padding': '5px'})],
        #            type="circle"),
        html.Div([dcc.Graph(id='playdata',figure=_manager.empty_view)],style={'padding': '5px'}),
    ],style={'width': '80%', 'display': 'inline-block', 'vertical-align': 'middle'})

    #
    # call backs
    #
    @app.callback(dash.dependencies.Output("wait_out", "children"), 
                  [dash.dependencies.Input("flash_or_hypo", "value")])
    def load_plib(flash_or_hypo):
        if flash_or_hypo == 'hypothesis':
            from flashmatch import phot
            phot.PhotonVisibilityService.GetME().LoadLibrary()
        return flash_or_hypo
    
    @app.callback(dash.dependencies.Output("select_qcluster","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def update_dropdown_select_qcluster(entry_or_event,data_index):
        """
        Update drop-down "select_qcluster" options for QCluster when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None or int(data_index) == _manager.current_data_index(is_entry=is_entry):
            raise dash.exceptions.PreventUpdate
        return _manager.dropdown_qcluster(data_index=int(data_index),is_entry=is_entry)
    
    @app.callback(dash.dependencies.Output("target_qcluster","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")])
    def update_dropdown_target_qcluster(entry_or_event,data_index):
        """
        Update drop-down "target_qcluster" options for QCluster when event/entry is changed 
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None or int(data_index) == _manager.current_data_index(is_entry=is_entry):
            raise dash.exceptions.PreventUpdate
        return _manager.dropdown_qcluster(data_index=int(data_index),is_entry=is_entry)
    
    @app.callback(dash.dependencies.Output("select_flash","options"),
                  [dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value"),
                   dash.dependencies.Input("flash_or_hypo","value")])
    def update_dropdown_flash(entry_or_event,data_index,flash_or_hypo):
        """
        Update drop-down "select_flash" options for Flash when event/entry is changed
        """
        is_entry = entry_or_event == 'entry'
        is_hypothesis = flash_or_hypo == 'hypothesis'
        if data_index is None or (int(data_index) == _manager.current_data_index(is_entry=is_entry)
                                  and not is_hypothesis):
            raise dash.exceptions.PreventUpdate
        return _manager.dropdown_flash(data_index=int(data_index),
                                       is_entry=is_entry,
                                       is_hypothesis=is_hypothesis)
    
    @app.callback(dash.dependencies.Output("visdata","figure"),
                  [dash.dependencies.Input("select_qcluster","value"),
                   dash.dependencies.Input("select_flash","value"),
                   dash.dependencies.Input("flash_or_hypo","value"),
                   dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")]
                 )
    def update_static(select_qcluster, select_flash, flash_or_hypo, entry_or_event, data_index):
        """
        Update "visdata" 3D display for change in event/entry and/or selection of flash/qcluster
        """
        is_entry = entry_or_event == 'entry'
        is_hypothesis = flash_or_hypo == 'hypothesis'
        if data_index is None: raise dash.exceptions.PreventUpdate
        return _manager.event_display(int(data_index), select_qcluster, select_flash, is_entry, is_hypothesis)
    
    @app.callback(dash.dependencies.Output("playdata","figure"),
                  [dash.dependencies.Input("target_qcluster","value"),
                   dash.dependencies.Input("xoffset","value"),
                   dash.dependencies.Input("entry_or_event","value"),
                   dash.dependencies.Input("data_index","value")]
                 )
    def update_dynamic(target_qcluster, xoffset, entry_or_event, data_index):
        """
        Update "playdata" 3D display for change in event/entry and/or selection of flash/qcluster
        """
        is_entry = entry_or_event == 'entry'
        if data_index is None or target_qcluster is None or len(target_qcluster)<1:
            raise dash.exceptions.PreventUpdate
        return _manager.hypothesis_display(int(data_index), target_qcluster, is_entry, float(xoffset))

    app.server.run()

In [ ]:
cfg = '../dat/flashmatch.cfg'
geo = '../dat/detector_specs.cfg'
data_particle = '../particleana_000.root'
data_opflash = '../opflashana_000.root'
view_data(cfg,geo,data_particle,data_opflash)